In [70]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from sklearn import datasets
from mlflow.models.signature import infer_signature
import time

# Load Iris dataset with direct X,y return
X, y = datasets.load_iris(return_X_y=True)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=8888
)

# Define base parameters shared between solvers
base_params = {
    "penalty": "l2",
    "max_iter": 1000,
    "multi_class": "multinomial",
    "random_state": 8888,
    "tol": 1e-4,
    "C": 1.0
}

# Define solvers to test
solvers = [
    {
        "name": "newton-cg",
        "params": {**base_params, "solver": "newton-cg"}
    },
    {
        "name": "bfgs",
        "params": {**base_params, "solver": "lbfgs"}
    }
]

# Set up MLflow tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Solver Comparison")

for solver in solvers:
    # Start MLflow run for each solver
    with mlflow.start_run(run_name=f"lr_{solver['name']}") as run:
        # Train the model and measure time
        start_time = time.time()
        lr = LogisticRegression(**solver['params'])
        lr.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        # Make predictions
        y_pred = lr.predict(X_test)
        
        # Calculate metrics
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "f1_score_macro": f1_score(y_test, y_pred, average='macro'),
            "precision_macro": precision_score(y_test, y_pred, average='macro'),
            "recall_macro": recall_score(y_test, y_pred, average='macro'),
            "training_time": training_time
        }
        
        # Log parameters
        mlflow.log_params(solver['params'])
        
        # Log metrics
        mlflow.log_metrics(metrics)
        
        # Set tags
        mlflow.set_tags({
            "training_info": f"{solver['name']} lr model",
            "data_source": "iris_dataset",
            "model_type": f"logistic_regression_{solver['name']}"
        })
        
        # Log model with signature
        signature = infer_signature(X_train, lr.predict(X_train))
        model_info = mlflow.sklearn.log_model(
            sk_model=lr,
            artifact_path=f"iris_model_{solver['name']}",
            signature=signature,
            input_example=X_train,
            registered_model_name=f"tracking-quickstart-{solver['name']}"
        )
        
        print(f"\nResults for {solver['name']} solver:")
        print(f"Run ID: {run.info.run_id}")
        print("Metrics:")
        for metric_name, metric_value in metrics.items():
            print(f"{metric_name}: {metric_value:.4f}")

print("\nAll solver comparisons completed successfully!")

2024/12/16 14:22:50 INFO mlflow.tracking.fluent: Experiment with name 'Solver Comparison' does not exist. Creating a new experiment.
c:\MLflow\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
Successfully registered model 'tracking-quickstart-newton-cg'.
2024/12/16 14:23:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart-newton-cg, version 1
Created version '1' of model 'tracking-quickstart-newton-cg'.



Results for newton-cg solver:
Run ID: 42b6cae8798b4565a052eb3bb6bcd692
Metrics:
accuracy: 0.9667
f1_score_macro: 0.9619
precision_macro: 0.9583
recall_macro: 0.9697
training_time: 0.0170
🏃 View run lr_newton-cg at: http://127.0.0.1:5000/#/experiments/551225698594173623/runs/42b6cae8798b4565a052eb3bb6bcd692
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/551225698594173623


c:\MLflow\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
Successfully registered model 'tracking-quickstart-bfgs'.
2024/12/16 14:23:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart-bfgs, version 1



Results for bfgs solver:
Run ID: 62bf9fd6cec9458ab8f865057512cadb
Metrics:
accuracy: 0.9667
f1_score_macro: 0.9619
precision_macro: 0.9583
recall_macro: 0.9697
training_time: 0.0446
🏃 View run lr_bfgs at: http://127.0.0.1:5000/#/experiments/551225698594173623/runs/62bf9fd6cec9458ab8f865057512cadb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/551225698594173623

All solver comparisons completed successfully!


Created version '1' of model 'tracking-quickstart-bfgs'.


In [71]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/62bf9fd6cec9458ab8f865057512cadb/iris_model_bfgs'

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      5.8,
      4.0,
      1.2,
      0.2
    ],
    [
      7.1,
      3.0,
      5.9,
      2.1
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      4.6,
      3.1,
      1.5,
      0.2
    ],
    [
      4.8,
      3.0,
      1.4,
      0.3
    ],
    [
      5.1,
      3.4,
      1.5,
      0.2
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      6.9,
      3.2,
      5.7,
      2.3
    ],
    [
      6.9,
      3.1,
      5.1,
      2.3
    ],
    [
      6.3,
      3.4,
      5.6,
      2.4
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      4.8,
      3.1,
      1.6,
      0.2
    ],
    [
      4.5,
      2.3,
      1.3,
      0.3
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      4.9,
      2.5,
      4.5,
      1.7
    ],
    [
      5.3,
      3.7,
      1.5,
      0.2
    ],
    [
      7.0,
      3.2,
      4.7,
      1.4
    ],
    [
      5.6,
      2.9,
      3.6,
      1.3
    ],
    [
      5.0,
      3.2,
      1.2,
      0.2
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      5.2,
      4.1,
      1.5,
      0.1
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      6.1,
      2.9,
      4.7,
      1.4
    ],
    [
      6.9,
      3.1,
      4.9,
      1.5
    ],
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      6.3,
      2.7,
      4.9,
      1.8
    ],
    [
      5.6,
      2.8,
      4.9,
      2.0
    ],
    [
      5.9,
      3.0,
      5.1,
      1.8
    ],
    [
      7.3,
      2.9,
      6.3,
      1.8
    ],
    [
      5.4,
      3.9,
      1.7,
      0.4
    ],
    [
      5.1,
      3.8,
      1.5,
      0.3
    ],
    [
      7.4,
      2.8,
      6.1,
      1.9
    ],
    [
      5.7,
      2.9,
      4.2,
      1.3
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      5.0,
      3.3,
      1.4,
      0.2
    ],
    [
      6.7,
      3.0,
      5.2,
      2.3
    ],
    [
      6.4,
      2.8,
      5.6,
      2.1
    ],
    [
      7.6,
      3.0,
      6.6,
      2.1
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      7.7,
      3.0,
      6.1,
      2.3
    ],
    [
      5.1,
      3.3,
      1.7,
      0.5
    ],
    [
      6.5,
      3.0,
      5.8,
      2.2
    ],
    [
      5.1,
      3.5,
      1.4,
      0.3
    ],
    [
      6.1,
      2.6,
      5.6,
      1.4
    ],
    [
      5.4,
      3.9,
      1.3,
      0.4
    ],
    [
      4.6,
      3.2,
      1.4,
      0.2
    ],
    [
      6.1,
      2.8,
      4.0,
      1.3
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      5.8,
      2.6,
      4.0,
      1.2
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      7.7,
      2.8,
      6.7,
      2.0
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      6.5,
      3.2,
      5.1,
      2.0
    ],
    [
      6.3,
      3.3,
      4.7,
      1.6
    ],
    [
      4.8,
      3.4,
      1.9,
      0.2
    ],
    [
      4.4,
      2.9,
      1.4,
      0.2
    ],
    [
      6.3,
      2.3,
      4.4,
      1.3
    ],
    [
      5.7,
      2.8,
      4.5,
      1.3
    ],
    [
      6.3,
      2.5,
      5.0,
      1.9
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      4.8,
      3.0,
      1.4,
      0.1
    ],
    [
      6.0,
      3.4,
      4.5,
      1.6
    ],
    [
      5.0,
      2.3,
      3.3,
      1.0
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      5.5,
      3.5,
      1.3,
      0.2
    ],
    [
      6.1,
      3.0,
      4.6,
      1.4
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      5.7,
      3.8,
      1.7,
      0.3
    ],
    [
      5.6,
      3.0,
      4.5,
      1.5
    ],
    [
      5.2,
      3.5,
      1.5,
      0.2
    ],
    [
      4.7,
      3.2,
      1.6,
      0.2
    ],
    [
      7.2,
      3.0,
      5.8,
      1.6
    ],
    [
      6.8,
      3.0,
      5.5,
      2.1
    ],
    [
      4.6,
      3.6,
      1.0,
      0.2
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      5.0,
      3.0,
      1.6,
      0.2
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      6.3,
      2.8,
      5.1,
      1.5
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      7.2,
      3.2,
      6.0,
      1.8
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      4.9,
      2.4,
      3.3,
      1.0
    ],
    [
      6.4,
      2.7,
      5.3,
      1.9
    ],
    [
      5.5,
      2.4,
      3.8,
      1.1
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      5.0,
      2.0,
      3.5,
      1.0
    ],
    [
      6.9,
      3.1,
      5.4,
      2.1
    ],
    [
      4.3,
      3.0,
      1.1,
      0.1
    ],
    [
      5.1,
      3.8,
      1.9,
      0.4
    ],
    [
      6.8,
      2.8,
      4.8,
      1.4
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      6.6,
      2.9,
      4.6,
      1.3
    ],
    [
      5.5,
      2.6,
      4.4,
      1.2
    ],
    [
      5.8,
      2.7,
      4.1,
      1.0
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      6.2,
      2.2,
      4.5,
      1.5
    ],
    [
      4.9,
      3.6,
      1.4,
      0.1
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.3,
      3.3,
      6.0,
      2.5
    ],
    [
      5.9,
      3.2,
      4.8,
      1.8
    ],
    [
      5.1,
      3.5,
      1.4,
      0.2
    ],
    [
      7.7,
      2.6,
      6.9,
      2.3
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      6.0,
      2.7,
      5.1,
      1.6
    ],
    [
      5.1,
      2.5,
      3.0,
      1.1
    ],
    [
      5.7,
      2.6,
      3.5,
      1.0
    ],
    [
      5.1,
      3.7,
      1.5,
      0.4
    ],
    [
      7.2,
      3.6,
      6.1,
      2.5
    ],
    [
      5.6,
      3.0,
      4.1,
      1.3
    ],
    [
      6.6,
      3.0,
      4.4,
      1.4
    ],
    [
      5.2,
      2.7,
      3.9,
      1.4
    ],
    [
      6.1,
      2.8,
      4.7,
      1.2
    ],
    [
      6.4,
      3.2,
      4.5,
      1.5
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      6.1,
      3.0,
      4.9,
      1.8
    ],
    [
      6.3,
      2.5,
      4.9,
      1.5
    ],
    [
      7.9,
      3.8,
      6.4,
      2.0
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([2, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 1, 2, 0, 2, 2, 2, 1, 2, 0, 2, 0,
       2, 0, 0, 1, 1, 1, 2, 2, 1, 2, 1, 0, 0, 1, 1, 2, 2, 0, 1, 1, 1, 0,
       1, 2, 0, 1, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 1, 2, 1, 1, 1,
       2, 0, 0, 1, 2, 0, 1, 1, 1, 2, 1, 0, 2, 2, 2, 0, 2, 2, 2, 1, 1, 0,
       2, 1, 1, 1, 1, 1, 0, 2, 1, 2])

In [72]:
# Load the saved model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

# Make predictions
predictions = loaded_model.predict(X_test)

# Get feature names
iris_feature_names = datasets.load_iris().feature_names

# Create results DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,5.6,2.7,4.2,1.3,1,1
1,4.7,3.2,1.3,0.2,0,0
2,6.7,3.0,5.0,1.7,1,2
3,6.0,3.0,4.8,1.8,2,2
4,6.7,2.5,5.8,1.8,2,2
5,5.0,3.4,1.5,0.2,0,0
6,6.5,2.8,4.6,1.5,1,1
7,6.0,2.2,5.0,1.5,2,2
8,5.7,3.0,4.2,1.2,1,1
9,6.7,3.1,5.6,2.4,2,2
